![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForMultipleChoice.ipynb)

## Import ONNX BertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `BertForMultipleChoice` is only available since in `Spark NLP 5.5.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `BertForMultipleChoice` or `TFBertForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [BertForMultipleChoice](https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 onnx optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00


- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the treained model above as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.
- In addition to the BERT model, we also need to save the `BertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "irfanamal/bert_multiple_choice"
EXPORT_PATH = f"onnx_models/bert_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

('onnx_models/bert_multiple_choice/tokenizer_config.json',
 'onnx_models/bert_multiple_choice/special_tokens_map.json',
 'onnx_models/bert_multiple_choice/vocab.txt',
 'onnx_models/bert_multiple_choice/added_tokens.json',
 'onnx_models/bert_multiple_choice/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -lh {EXPORT_PATH}

total 419M
-rw-r--r-- 1 root root  683 Jun  8 14:19 config.json
-rw-r--r-- 1 root root 418M Jun  8 14:19 model.onnx
-rw-r--r-- 1 root root  695 Jun  8 14:20 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Jun  8 14:20 tokenizer_config.json
-rw-r--r-- 1 root root 695K Jun  8 14:20 tokenizer.json
-rw-r--r-- 1 root root 227K Jun  8 14:20 vocab.txt


- Now we need to move `vocab.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find it.

In [ ]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [ ]:
!ls -lR {EXPORT_PATH}

onnx_models/bert_multiple_choice:
total 428680
drwxr-xr-x 2 root root      4096 Jun  8 14:22 assets
-rw-r--r-- 1 root root       683 Jun  8 14:19 config.json
-rw-r--r-- 1 root root 438238205 Jun  8 14:19 model.onnx
-rw-r--r-- 1 root root       695 Jun  8 14:20 special_tokens_map.json
-rw-r--r-- 1 root root      1328 Jun  8 14:20 tokenizer_config.json
-rw-r--r-- 1 root root    711494 Jun  8 14:20 tokenizer.json

onnx_models/bert_multiple_choice/assets:
total 228
-rw-r--r-- 1 root root 231508 Jun  8 14:20 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save BertForMultipleChoice in Spark NLP

Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [ ]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 44.3 MB/s eta 0:00:00


In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import BertForMultipleChoice

bertMultpleChoiceClassifier = BertForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [ ]:
bertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

In [ ]:
! ls -l /content/irfanamal/bert_multiple_choice_spark_nlp_onnx

total 428048
-rw-r--r-- 1 root root 438305209 Jun  8 14:27 bert_mc_classification_onnx
drwxr-xr-x 3 root root      4096 Jun  8 14:27 fields
drwxr-xr-x 2 root root      4096 Jun  8 14:27 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [ ]:
testing_data = [
    ("In Italy, pizza served in formal settings is presented unsliced.",
     "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against which group?", "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygen, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
]

testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+-----------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                         |choices                                                                 |
+-----------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings is presented unsliced. |It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                    |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                 |Lion, Elephant, Tiger, Leopard                                          |
|Water boils at what temperature?                                 |90°C, 120°C, 100°C                                                      |
|Which planet

In [ ]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import BertForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

bert_for_multiple_choice = BertForMultipleChoice() \
    .load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    bert_for_multiple_choice
])

pipeline_model = pipeline.fit(testing_df)
pipeline_df = pipeline_model.transform(testing_df)
pipeline_df.select("answer.result").show(truncate=False)

+-------------------------------------+
|result                               |
+-------------------------------------+
|[It is eaten with a fork and a knife]|
|[Germany]                            |
|[Lion]                               |
|[90°C]                               |
|[ Mars]                              |
|[ Portuguese]                        |
|[ The Mongols]                       |
|[Oxygen]                             |
|[ Africa]                            |
|[Vincent van Gogh]                   |
+-------------------------------------+

